## Imports

In [3]:
import sys
import random
from pathlib import Path
from collections import Counter

import datasets
from datasets import load_dataset_builder, load_dataset

from utils import replace_text_with_whitespace

In [4]:
ELK_PATH = Path("../../../elk/")
ELK_PATH.resolve()

PosixPath('/fsx/home-augustas/elk')

In [5]:
modules = [
    ELK_PATH,
    ELK_PATH / "elk" / "promptsource",
]

for module in modules:
    if not str(module) in sys.path:
        sys.path.insert(0, str(module.resolve()))

sys.path[:2]

['/fsx/home-augustas/elk/elk/promptsource', '/fsx/home-augustas/elk']

In [6]:
from templates import DatasetTemplates

In [7]:
# Disable the logging of the datasets library
import datasets

datasets.logging.set_verbosity_error()

## Config

In [14]:
# all datasets used in Burns et al. (2022) (apart from story_cloze)
BURNS_DATASETS = [
    "ag_news",
    "amazon_polarity",
    "dbpedia_14",
    "glue:qnli",
    "imdb",
    "piqa",
    "super_glue:boolq",
    "super_glue:copa",
    "super_glue:rte",
]
# BURNS_DATASETS = ["ag_news"]
# BURNS_DATASETS = [
#     "ag_news",
#     "amazon_polarity",
#     "dbpedia_14",
#     "glue:qnli",
#     "imdb",
#     "super_glue:boolq",
#     "super_glue:copa",
#     "super_glue:rte",
# ]
# BURNS_DATASETS = [
#     "ag_news",
#     "amazon_polarity",
#     "dbpedia_14",
#     "imdb",
#     "super_glue:boolq",
#     "super_glue:copa",
# ]

VERSION = "v4"

# SPLIT = "validation"
SPLIT = "train"

# These numbers are chosen so that both datasets have
# approximately 10k examples in total (probably a bit less for train split)
# N_PER_DATASET = 1000 if SPLIT == "train" else 1000
# N_PER_DATASET = 1200 if SPLIT == "train" else 1375
N_PER_DATASET = 1200 if SPLIT == "train" else 1800
if SPLIT == "train":
    assert N_PER_DATASET <= 2490, "N_PER_DATASET must be <= 2490"
else:
    assert N_PER_DATASET <= 1838, "N_PER_DATASET must be <= 1838"

SEED = 42 if SPLIT == "train" else 2023
SEED

42

## Load and inspect the datasets

In [15]:
dataset_dict = {}
for dataset_path in BURNS_DATASETS:
    print(dataset_path)

    # Parse dataset name
    dataset_name = None    
    if ":" in dataset_path:
        dataset_path, dataset_name = dataset_path.split(":")
    
    
    # Get the most validation-like split
    available_splits = load_dataset_builder(
        dataset_path, name=dataset_name
    ).info.splits.keys()
    split = "validation" if "validation" in available_splits else "test"
    split = split if SPLIT != "train" else "train"
    print(split)

    # Load the dataset
    dataset = load_dataset(
        dataset_path, name=dataset_name, split=split,
    )

    # Get a desired subset of the data
    n = N_PER_DATASET if dataset.num_rows > N_PER_DATASET else dataset.num_rows
    dataset = dataset.shuffle(seed=SEED).select(range(n))

    key = f"{dataset_path}/{dataset_name}" if dataset_name else dataset_path
    dataset_dict[key] = dataset

    print("-----------------------------------")

ag_news


train
-----------------------------------
amazon_polarity
train
-----------------------------------
dbpedia_14
train
-----------------------------------
glue:qnli
train
-----------------------------------
imdb
train
-----------------------------------
piqa
train
-----------------------------------
super_glue:boolq
train
-----------------------------------
super_glue:copa
train
-----------------------------------
super_glue:rte
train
-----------------------------------


In [16]:
sum(dataset.num_rows for dataset in dataset_dict.values())

10000

In [17]:
assert all([len(Counter(dataset["label"])) > 1 for dataset in dataset_dict.values()])

In [18]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {dataset.num_rows}")

ag_news: 1200
amazon_polarity: 1200
dbpedia_14: 1200
glue/qnli: 1200
imdb: 1200
piqa: 1200
super_glue/boolq: 1200
super_glue/copa: 400
super_glue/rte: 1200


In [19]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {len(Counter(dataset['label']))}")

ag_news: 4
amazon_polarity: 2
dbpedia_14: 14
glue/qnli: 2
imdb: 2
piqa: 2
super_glue/boolq: 2
super_glue/copa: 2
super_glue/rte: 2


## Get the templates

In [20]:
dataset_template_dict = {}

for dataset_path in dataset_dict.keys():
    dataset_templates = DatasetTemplates(dataset_path)

    dataset_templates.templates = {
        x.name: x for x in dataset_templates.templates.values() if x.get_answer_choices_list(dataset_dict[dataset_path][0]) is not None
    }

    dataset_template_dict[dataset_path] = dataset_templates

In [21]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 15
amazon_polarity: 11
dbpedia_14: 11
glue/qnli: 5
imdb: 13
piqa: 7
super_glue/boolq: 10
super_glue/copa: 9
super_glue/rte: 11


In [22]:
dataset_name = "ag_news"
for idx, (template_name, template) in enumerate(dataset_template_dict[dataset_name].templates.items()):
    print(f"{idx+1}) {dataset_name}/{template_name}: {template.metadata.choices_in_prompt}")


1) ag_news/classify_question_first: False
2) ag_news/classify_with_choices_question_first: True
3) ag_news/recommend: True
4) ag_news/which_section_choices: True
5) ag_news/which_section: False
6) ag_news/burns_1: True
7) ag_news/burns_2: True
8) ag_news/burns_3: True
9) ag_news/burns_4: True
10) ag_news/burns_5: True
11) ag_news/burns_6: True
12) ag_news/burns_7: True
13) ag_news/burns_8: True
14) ag_news/classify_with_choices: True
15) ag_news/classify: False


In [23]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    good_templates = [x for x in dataset_templates.templates.values() if x.metadata.choices_in_prompt]
    print(f"{dataset_name}: {len(good_templates)}")

ag_news: 12
amazon_polarity: 8
dbpedia_14: 11
glue/qnli: 1
imdb: 4
piqa: 6
super_glue/boolq: 5
super_glue/copa: 9
super_glue/rte: 11


## Inspect the prompts

In [24]:
# dataset_name = "super_glue/rte"
# dataset_name = "glue/qnli"

# for template_name, template in dataset_template_dict[dataset_name].templates.items():
#     # print(template_name)
#     # print(dataset_dict[dataset_name][0])
#     q, a = template.apply(
#         dataset_dict[dataset_name][1]
#     )
#     # print(q == q.strip())
#     # print(a == a.strip())
#     # print(" ".join([q, a.strip()]))
#     print(" ".join([q, a]))
#     print(len(a))
#     print("---------------------------------")


## Form the dataset for the chosen split

In [25]:
# for dataset_name, dataset_templates in dataset_template_dict.items():
#     print(dataset_name)
#     for template_name, template in dataset_templates.templates.items():
#         print(f"{template_name}: {template.get_fixed_answer_choices_list()}")

#     print("---------------------------------")


In [26]:
# for dataset_name, dataset_templates in dataset_template_dict.items():
#     print(dataset_name)
#     for template_name, template in dataset_templates.templates.items():
#         print(f"{template_name}: {template.get_answer_choices_list(dataset_dict[dataset_name][0])}")

#     print("---------------------------------")

In [27]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 15
amazon_polarity: 11
dbpedia_14: 11
glue/qnli: 5
imdb: 13
piqa: 7
super_glue/boolq: 10
super_glue/copa: 9
super_glue/rte: 11


In [28]:
%%time

# Reproducibility
random.seed(SEED)

ALLOWED_KEYS = ["text", "label", "original_dataset", "template_name"]

new_dataset = []

for dataset_name, dataset in dataset_dict.items():
    print(dataset_name)

    for idx, entry in enumerate(dataset):
        new_entry = entry.copy()
        
        # In case we need to know which dataset the entry came from
        new_entry["original_dataset"] = dataset_name

        # Sample a random template
        template_name = random.choice(
            list(dataset_template_dict[dataset_name].templates.keys())
        )
        template = dataset_template_dict[dataset_name].templates[template_name]
        new_entry["template_name"] = template_name

        # Whether the sample will be truthful or not
        is_truthful = random.choice([True, False])

        # If not truthful, sample a random incorrect label
        if not is_truthful and len(dataset.features["label"].names) > 2:
            label_mappping = dataset.features["label"] # [class 1, class 2, ...]
            all_label_ids = [label_mappping.str2int(x) for x in label_mappping.names] # [0, 1, ...]
            incorrect_label_id = all_label_ids[(1 - new_entry["label"]) % len(all_label_ids)]
            incorrect_label = template.get_fixed_answer_choices_list()[incorrect_label_id]
            
        
        # Apply the template
        if is_truthful:
            new_text = " ".join(template.apply(new_entry))
        elif not is_truthful and len(dataset.features["label"].names) > 2:
            q, a = template.apply(new_entry)
            incorrect_label = replace_text_with_whitespace(a, incorrect_label)
            new_text = " ".join([q, incorrect_label])
        else:
            # Untruthful binary case
            new_entry["label"] = 1 - new_entry["label"]
            new_text = " ".join(template.apply(new_entry))

        new_entry["text"] = new_text

        # We can now change the label to whether the sample is truthful or not
        new_entry["label"] = int(is_truthful)

        # Remove all other keys
        new_entry = { k: v for k, v in new_entry.items() if k in ALLOWED_KEYS }

        # Append to the new dataset
        new_dataset.append(new_entry)

ag_news
amazon_polarity
dbpedia_14
glue/qnli
imdb
piqa
super_glue/boolq
super_glue/copa
super_glue/rte
CPU times: user 20.9 s, sys: 529 ms, total: 21.4 s
Wall time: 28 s


In [29]:
my_dataset = datasets.Dataset.from_list(new_dataset)
my_dataset

Dataset({
    features: ['text', 'label', 'original_dataset', 'template_name'],
    num_rows: 10000
})

In [35]:
current_idx = 0
my_dataset[current_idx]

{'text': 'Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally. Which is the topic of this example, choice 1: World politics, or choice 2: Sports? World politics',
 'label': 1,
 'original_dataset': 'ag_news',
 'template_name': 'burns_6'}

In [36]:
print(my_dataset[current_idx]["text"])

Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally. Which is the topic of this example, choice 1: World politics, or choice 2: Sports? World politics


In [37]:
Counter(my_dataset["label"]), Counter(my_dataset["original_dataset"])

(Counter({1: 5010, 0: 4990}),
 Counter({'ag_news': 1200,
          'amazon_polarity': 1200,
          'dbpedia_14': 1200,
          'glue/qnli': 1200,
          'imdb': 1200,
          'piqa': 1200,
          'super_glue/boolq': 1200,
          'super_glue/rte': 1200,
          'super_glue/copa': 400}))

In [39]:
# my_dataset.to_parquet(f"datasets/burns_datasets_VINC_{SPLIT}_{VERSION}.parquet")

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

5636293

In [40]:
!ls -lah datasets | grep {VERSION}

-rw-r--r-- 1 augustas Domain Users 2.8M Jun 29 10:48 burns_datasets_VINC_train_v4.parquet


In [63]:
# parquet_data_files = {
#     "train": "datasets/my_dataset_train.parquet",
#     "validation": "datasets/my_dataset_validation.parquet",
# }

# my_dataset = load_dataset("parquet", data_files=parquet_data_files)
# my_dataset